Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin

SEED = 21

PATH = '/datasets/'

In [2]:
data = pd.read_csv(os.path.join(PATH, 'insurance.csv'))
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
X = data.drop(columns='Страховые выплаты')
y = data['Страховые выплаты']

X.shape, y.shape

((5000, 4), (5000,))

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

> Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

Попробую переформулировать вопрос

Если в задаче линеной регрессии $a = Xw$ умножить $X (M, N)$ на обратимую матрицу $Z$, размерности $(N, N)$, то при вычислении весов $w = (X^T X)^{-1} X^T y$ предсказания $a$ не дожны изменится.



**Ответ:** Предсказания $a$ не изменятся

**Обоснование:**

Для доказательства подставим вместо $X$ $XZ$

$$
a_{new} = XZw_{new}
$$

$$
w_{new} = ((XZ)^T XZ)^{-1}(XZ)^T y
$$

подставим $w_{new}$ в $a_{new}$ и преобразуем:

$$
a_{new} = X Z ((XZ)^T XZ)^{-1}(XZ)^T y = \\X Z (Z^TX^T XZ)^{-1}Z^T X^T y = \\X Z (X^TXZ)^{-1} (Z^T)^{-1} Z^T X^T y = \\X Z Z^{-1}(X^TX)^{-1} (Z^T)^{-1} Z^T X^T y = \\X E (X^TX)^{-1} E  X^T y = \\X (X^TX)^{-1} X^T y =  X w
$$

Пояснения:
1. Раскрываем транспонирования по свойству: $(AB)^T = B^T A^T$
2. Считаем $X^TX$ - квадратная обратимая марица, $Z$ - обратимая по опеределнию, то раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ 2 раза
3. Используя ассоциативность умножения матриц и определение обратной матрицы получим ответ

Таким образом получаем что:

$$
a_{new} = a
$$

P.S. во всех этих рассуждениях одно допущение, что $X^TX$ - обратима, но в общем случае она может оказаться не обратимой

## 3. Алгоритм преобразования

**Алгоритм**

Не очень понял что тут надо написать, доказательство выше, реализация ниже :)

**Обоснование**

...

## 4. Проверка алгоритма

Реализуем класс для получения новых признаков, умноженных на матрицу $Z$

Проверим размерности выходных данных

In [5]:
class FeatPreparer(TransformerMixin, BaseEstimator):
    
    
    def fit(self, X, y=None, random_state=None):
        if random_state:
            np.random.seed(random_state)
            
        X_dim = X.shape[1]
        Z = np.random.normal(size=(X_dim, X_dim))
        
        
        self.Z = Z
        self.X = X
        return self
        
    
    def transform(self, X, y=None):  
        '''
        >>> preparer = FeatPreparer()
        >>> X = np.random.normal(size=(100,5))
        >>> X_new = preparer.fit_transform(X)
        >>> X_new.shape == X.shape
        True
        '''
        return X.dot(self.Z)  
    

In [6]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

In [7]:
X_new = FeatPreparer().fit_transform(X, random_state=SEED)
X.shape, X_new.shape

((5000, 4), (5000, 4))

1. Обучим 2 линейные регрессии
1. Сделаем предсказания на обучающей выборке
1. Посчитаем метрики и R2 и сравним между собой

In [8]:
linreg = LinearRegression()

linreg.fit(X, y)
predict1 = linreg.predict(X)

linreg.fit(X_new, y)
predict2 = linreg.predict(X_new)

np.isclose(r2_score(y, predict1), r2_score(y, predict2))

True

Как мы видим метрики одинаковые, это подтверждает теоретический вывод.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования